In [1]:
from MBDOE import *
from generalize_functions import CEOSModels


[    0.00] Initializing mpi-sppy


## Choose experiments and model

In [2]:
model_option = "PR"
if_poly = False

if model_option=="PR":
    if not if_poly:
        from hfc32_emimtf2n_PR import configuration
    else:
        from hfc32_emimtf2n_PR_polynomial import configuration
        
elif model_option=="SRK":
    if not if_poly:
        from hfc32_emimtf2n_SRK import configuration
    else:
        from hfc32_emimtf2n_SRK_polynomial import configuration
        

In [3]:
# Experiments 
data_file = './emimtf2n/R32/Final_Results/MBDoE/r32_emimtf2n_subset.csv'

path = './emimtf2n/R32/Final_Results/MBDoE/Params/'

# Decide on model 
#para_file = path + 'PR_params_1param_Opt1.csv'
#PR_type_opt = "1Param_Opt1"

#para_file = path + 'PR_params_1param_Opt2.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'PR_params_1param_Opt2_GF.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'PR_params_3params_Opt1.csv'
#PR_type_opt = "3Params_Opt1"

#para_file = path + 'PR_params_3params_Opt2.csv'
#PR_type_opt = "3Params_Opt2"

#para_file = path + 'PR_params_linTdep.csv'
#PR_type_opt = "Linear"

#para_file = path + 'PR_params_noTdep.csv'
#PR_type_opt = "No"

#para_file = path + 'PR_params_polyTdep.csv'
#PR_type_opt = "Polynomial"

# important model
para_file = path + 'PR_params_quadTdep.csv'
PR_type_opt = "Quadratic"

#para_file = path + 'SRK_params_1param_Opt1.csv'
#PR_type_opt = "1Param_Opt1"

#para_file = path + 'SRK_params_1param_Opt2.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'SRK_params_3params_Opt1.csv'
#PR_type_opt = "3Params_Opt1"

#para_file = path + 'SRK_params_3params_Opt2.csv'
#PR_type_opt = "3Params_Opt2"

#para_file = path + 'SRK_params_linTdep.csv'
#PR_type_opt = "Linear"

#para_file = path + 'SRK_params_noTdep.csv'
#PR_type_opt = "No"


#para_file = path + 'SRK_params_polyTdep.csv'
#PR_type_opt = "Polynomial"

#para_file = path + 'SRK_params_quadTdep.csv'
#PR_type_opt = "Quadratic"



model_creation = CEOSModels(para_file, configuration, 
                         comp_1= "R32", comp_2 = "emimTf2N", 
                         x_comp_1="x_R32", x_comp_2="x_emimTf2N",  Model_type = model_option, Tdep_type=PR_type_opt)

## Run pyomo.doe

In [ ]:
data_exp = pd.read_csv(data_file)
print(len(data_exp))
print(data_exp.iloc[1])

In [ ]:
print(model_creation.param_name_dict)

In [ ]:
mbdoe_obj = MBDOE(data_file, model_creation)

exp_set = range(len(data_exp))
#exp_set = range(12)
#exp_set = [0,1,2,3]

#exp_set = 0
# init_T: 283-348; 323 is good; init_pressure: low mole fraction use 150000; high 399800; x_option: 0.4-0.6;0.3-0.55;
# SRK: init_temp_option=323, init_pressure_option= 399800, init_x_c1_option = 0.5

total_fim = mbdoe_obj.sumDOE(exp_set, scale_opt=True, record_name="SRK_polyTdep", 
                             init_temp_option=343, 
                             init_pressure_option= 150000,
                             init_x_c1_option = 0.4,
                             poly_option=True)
#total_fim = mbdoe_obj.doe(exp_set, scale=True)

In [ ]:
print(total_fim)
print(np.linalg.det(total_fim))
print(np.trace(total_fim))
print(np.linalg.eigvals(total_fim))

## Heatmap

In [4]:
# read prior

f = open('emimtf2n_FIM_info/PR_quadTdep.json')

data = json.load(f)

prior_info = data[0]['Total']

print(prior_info)
print(np.log10(np.linalg.det(prior_info)))

[[45575355309926.85, -10796186966711.326, 83445290175042.88, -7989512603441.533, 30035490945132.06, -5294333373880.846], [-10796186966711.326, 3888813365343.9434, -19011816632190.887, 2790548602457.38, -6577442271879.425, 1789815131395.5442], [83445290175042.88, -19011816632190.887, 153592012364861.78, -14134663295880.395, 55573688924934.195, -9411789134390.35], [-7989512603441.533, 2790548602457.38, -14134663295880.395, 2008060912146.0166, -4913769119004.827, 1291821973754.2095], [30035490945132.06, -6577442271879.425, 55573688924934.195, -4913769119004.827, 20211335904074.926, -3288302607352.0684], [-5294333373880.846, 1789815131395.5442, -9411789134390.35, 1291821973754.2095, -3288302607352.0684, 833744206456.5271]]
61.06068485550636


In [ ]:
#data_exp = pd.read_csv(data_file)
#print(data_exp)

In [5]:
# model list 
#T_range = [273, 360]
#x_range = [0.1, 0.9]

T_range = [273, 283, 293, 303, 313, 323, 333, 343, 353, 363, 373, 383, 393, 400]
x_range = [0.1,0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

data_exp = {'pressure': 549300}

fixed_model_set = []

for i in range(len(T_range)):
    for j in range(len(x_range)):
        data_exp['temperature'] = T_range[i]
        data_exp['x_R32'] = x_range[j]
        data_exp['x_emimTf2N'] = 1-x_range[j]
    
        print(data_exp)
        
        model = model_creation.create_model(data_exp)
        
        fixed_model_set.append(model)

{'pressure': 549300, 'temperature': 273, 'x_R32': 0.1, 'x_emimTf2N': 0.9}
2022-10-20 00:42:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.2, 'x_emimTf2N': 0.8}
2022-10-20 00:42:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.3, 'x_emimTf2N': 0.7}
2022-10-20 00:42:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.4, 'x_emimTf2N': 0.6}
2022-10-20 00:42:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.5, 'x_emimTf2N': 0.5}
2022-10-20 00:42:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'tem

2022-10-20 00:43:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.7, 'x_emimTf2N': 0.30000000000000004}
2022-10-20 00:43:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.8, 'x_emimTf2N': 0.19999999999999996}
2022-10-20 00:43:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.9, 'x_emimTf2N': 0.09999999999999998}
2022-10-20 00:43:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 323, 'x_R32': 0.1, 'x_emimTf2N': 0.9}
2022-10-20 00:43:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 323, 'x_R32': 0

2022-10-20 00:43:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 363, 'x_R32': 0.3, 'x_emimTf2N': 0.7}
2022-10-20 00:43:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 363, 'x_R32': 0.4, 'x_emimTf2N': 0.6}
2022-10-20 00:43:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 363, 'x_R32': 0.5, 'x_emimTf2N': 0.5}
2022-10-20 00:43:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 363, 'x_R32': 0.6, 'x_emimTf2N': 0.4}
2022-10-20 00:43:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 363, 'x_R32': 0.7, 'x_emimTf2N': 0.30000000000000004}
2022-10-2

2022-10-20 00:43:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 400, 'x_R32': 0.8, 'x_emimTf2N': 0.19999999999999996}
2022-10-20 00:43:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 400, 'x_R32': 0.9, 'x_emimTf2N': 0.09999999999999998}
2022-10-20 00:44:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.


In [6]:
design_range_values = [T_range, x_range]

In [7]:
mbdoe_obj = MBDOE(data_file, model_creation)

all_fim = mbdoe_obj.run_grid_search(design_range_values, fixed_models=fixed_model_set, 
                                    prior_FIM = prior_info, 
                                    scale_opt=True,
                                   store_name="PR_quad_543900")
                                    #, record_name="SRK_polyTdep", 
                                    # init_temp_option=343, 
                                    # init_pressure_option= 150000,
                                    # init_x_c1_option = 0.4,
                                    # poly_option=True)



parameter set: {'fs.properties.PR_kappa_A["R32", "emimTf2N"]': 0.3212980565927437, 'fs.properties.PR_kappa_A["emimTf2N", "R32"]': 0.8429816989080334, 'fs.properties.PR_kappa_B["R32", "emimTf2N"]': -0.5532108602621684, 'fs.properties.PR_kappa_B["emimTf2N", "R32"]': -0.6100855368792957, 'fs.properties.PR_kappa_C["R32", "emimTf2N"]': 0.1862684932461356, 'fs.properties.PR_kappa_C["emimTf2N", "R32"]': -0.3935451934926509}
All measurements are flattened.
Flatten measurement name: ['fs.state_block.pressure']
126  design vectors will be searched.
=======This is the  1 th iteration=======
Design variable values of this iteration: {'fs.F101.inlet.temperature': {0: 1}, 'fs.F101.inlet.pressure': {0: 1}, 'fs.F101.inlet.mole_frac_comp[0,"R32"]': {0: 1}, "fs.F101.inlet.mole_frac_comp[0,'emimTf2N']": {0: 1}}
Sensitivity information is scaled by its corresponding parameter nominal value.
After practice: {'fs.state_block.pressure': [0]}
name: fs.state_block.pressure
Build time with direct kaug mode [s]:

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 7.3909759521484375e-06
Solve time with direct kaug mode [s]: 0.7059400081634521
Total wall clock time [s]: 0.7092373371124268
Existed information has been added.
FIM: [[ 4.63095095e+13 -1.09977334e+13  8.46028351e+13 -8.12305431e+12
   3.03923975e+13 -5.37319942e+12]
 [-1.09977334e+13  3.94414365e+12 -1.93295960e+13  2.82720964e+12
  -6.67542338e+12  1.81146613e+12]
 [ 8.46028351e+13 -1.93295960e+13  1.55417120e+14 -1.43452192e+13
   5.61364254e+13 -9.53613766e+12]
 [-8.12305431e+12  2.82720964e+12 -1.43452192e+13  2.03235198e+12
  -4.97868996e+12  1.30616761e+12]
 [ 3.03923975e+13 -6.67542338e+12  5.61364254e+13 -4.97868996e+12
   2.03848448e+13 -3.32664307e+12]
 [-5.37319942e+12  1.81146613e+12 -9.53613766e+12  1.30616761e+12
  -3.32664307e+12  8.42216341e+11]]
Trace: 228930185958057.0
Determinant: 1.7231013305604597e+61
Condition number: 260804823.602141
Minimal eigen value: 867197.3108821736
Eigen values: [2.261692

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 6.67572021484375e-06
Solve time with direct kaug mode [s]: 0.747711181640625
Total wall clock time [s]: 0.7485299110412598
Existed information has been added.
FIM: [[ 4.58893995e+13 -1.12354961e+13  8.39404665e+13 -8.28060852e+12
   3.01881755e+13 -5.46625611e+12]
 [-1.12354961e+13  4.50335272e+12 -1.97045074e+13  3.19775595e+12
  -6.79102920e+12  2.03031387e+12]
 [ 8.39404665e+13 -1.97045074e+13  1.54372793e+14 -1.45936555e+13
   5.58144378e+13 -9.68287227e+12]
 [-8.28060852e+12  3.19775595e+12 -1.45936555e+13  2.27788550e+12
  -5.05529653e+12  1.45118175e+12]
 [ 3.01881755e+13 -6.79102920e+12  5.58144378e+13 -5.05529653e+12
   2.02855694e+13 -3.37188942e+12]
 [-5.46625611e+12  2.03031387e+12 -9.68287227e+12  1.45118175e+12
  -3.37188942e+12  9.27862908e+11]]
Trace: 228256863001733.3
Determinant: 4.82501178588089e+61
Condition number: 155230821.55467254
Minimal eigen value: 1448342.9646132595
Eigen values: [2.24827468

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 6.9141387939453125e-06
Solve time with direct kaug mode [s]: 0.7497437000274658
Total wall clock time [s]: 0.7505941390991211
Existed information has been added.
FIM: [[ 4.64074145e+13 -1.09548073e+13  8.48052626e+13 -8.09845596e+12
   3.04701734e+13 -5.36102541e+12]
 [-1.09548073e+13  3.91905208e+12 -1.92710761e+13  2.81131711e+12
  -6.66030833e+12  1.80252902e+12]
 [ 8.48052626e+13 -1.92710761e+13  1.55814841e+14 -1.43127275e+13
   5.62841627e+13 -9.52079501e+12]
 [-8.09845596e+12  2.81131711e+12 -1.43127275e+13  2.02232511e+12
  -4.97068306e+12  1.30055411e+12]
 [ 3.04701734e+13 -6.66030833e+12  5.62841627e+13 -4.97068306e+12
   2.04384217e+13 -3.32314371e+12]
 [-5.36102541e+12  1.80252902e+12 -9.52079501e+12  1.30055411e+12
  -3.32314371e+12  8.39089774e+11]]
Trace: 229441144406966.4
Determinant: 1.4250379075638854e+61
Condition number: 275012605.50040275
Minimal eigen value: 824285.5245761025
Eigen values: [2.2668

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 7.3909759521484375e-06
Solve time with direct kaug mode [s]: 0.7323236465454102
Total wall clock time [s]: 0.7334442138671875
Existed information has been added.
FIM: [[ 4.70393934e+13 -1.20680109e+13  8.58381911e+13 -8.86314362e+12
   3.08003179e+13 -5.82921834e+12]
 [-1.20680109e+13  4.99365905e+12 -2.10905527e+13  3.54948027e+12
  -7.24185479e+12  2.25447485e+12]
 [ 8.58381911e+13 -2.10905527e+13  1.57503096e+14 -1.55625719e+13
   5.68237624e+13 -1.02860333e+13]
 [-8.86314362e+12  3.54948027e+12 -1.55625719e+13  2.52938012e+12
  -5.37016199e+12  1.61100227e+12]
 [ 3.08003179e+13 -7.24185479e+12  5.68237624e+13 -5.37016199e+12
   2.06108885e+13 -3.56773143e+12]
 [-5.82921834e+12  2.25447485e+12 -1.02860333e+13  1.61100227e+12
  -3.56773143e+12  1.02916394e+12]]
Trace: 233705580720550.62
Determinant: 2.1599258746707234e+61
Condition number: 267064679.1981026
Minimal eigen value: 861103.583456671
Eigen values: [2.29970

...

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 5.9604644775390625e-06
Solve time with direct kaug mode [s]: 0.5658185482025146
Total wall clock time [s]: 0.5665109157562256
Existed information has been added.
FIM: [[ 4.66203144e+13 -1.08356909e+13  8.58171596e+13 -8.02716184e+12
   3.10882970e+13 -5.32631516e+12]
 [-1.08356909e+13  3.89030679e+12 -1.91014835e+13  2.79197191e+12
  -6.61724288e+12  1.79102418e+12]
 [ 8.58171596e+13 -1.91014835e+13  1.58975730e+14 -1.42201203e+13
   5.79633697e+13 -9.48438204e+12]
 [-8.02716184e+12  2.79197191e+12 -1.42201203e+13  2.00941739e+12
  -4.95170108e+12  1.29297426e+12]
 [ 3.10882970e+13 -6.61724288e+12  5.79633697e+13 -4.95170108e+12
   2.12720479e+13 -3.32052455e+12]
 [-5.32631516e+12  1.79102418e+12 -9.48438204e+12  1.29297426e+12
  -3.32052455e+12  8.34723034e+11]]
Trace: 233602539627407.1
Determinant: 1.13343371024069e+62
Condition number: 152948362.0247818
Minimal eigen value: 1508198.4833863252
Eigen values: [2.306764

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 5.7220458984375e-06
Solve time with direct kaug mode [s]: 0.5371761322021484
Total wall clock time [s]: 0.5378165245056152
Existed information has been added.
FIM: [[ 1.54262343e+14 -4.50045948e+13  3.30126887e+14 -4.06219882e+13
   1.39522365e+14 -3.30402119e+13]
 [-4.50045948e+13  1.46556511e+13 -9.66529846e+13  1.30613736e+13
  -4.10376081e+13  1.05226207e+13]
 [ 3.30126887e+14 -9.66529846e+13  7.13473296e+14 -8.81990140e+13
   3.04070748e+14 -7.23852670e+13]
 [-4.06219882e+13  1.30613736e+13 -8.81990140e+13  1.18057237e+13
  -3.77864045e+13  9.62231967e+12]
 [ 1.39522365e+14 -4.10376081e+13  3.04070748e+14 -3.77864045e+13
   1.30503982e+14 -3.12383779e+13]
 [-3.30402119e+13  1.05226207e+13 -7.23852670e+13  9.62231967e+12
  -3.12383779e+13  7.91677953e+12]]
Trace: 1032617776252189.0
Determinant: 6.18032241422887e+64
Condition number: 481179294.4252026
Minimal eigen value: 2134888.927019598
Eigen values: [1.02726435e

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 5.245208740234375e-06
Solve time with direct kaug mode [s]: 0.5684702396392822
Total wall clock time [s]: 0.5711679458618164
Existed information has been added.
FIM: [[ 8.74841179e+15 -9.46777774e+15  1.98355893e+16 -9.02955001e+15
   8.79668755e+15 -7.67611960e+15]
 [-9.46777774e+15  1.02803660e+16 -2.14827797e+16  9.80611522e+15
  -9.53290457e+15  8.33733117e+15]
 [ 1.98355893e+16 -2.14827797e+16  4.49834812e+16 -2.04896616e+16
   1.99525550e+16 -1.74192811e+16]
 [-9.02955001e+15  9.80611522e+15 -2.04896616e+16  9.35396526e+15
  -9.09262701e+15  7.95304579e+15]
 [ 8.79668755e+15 -9.53290457e+15  1.99525550e+16 -9.09262701e+15
   8.85114697e+15 -7.73036172e+15]
 [-7.67611960e+15  8.33733117e+15 -1.74192811e+16  7.95304579e+15
  -7.73036172e+15  6.76202714e+15]]
Trace: 8.897939829715744e+16
Determinant: 8.745160180817082e+67
Condition number: 20225518447.182213
Minimal eigen value: 4397146.971751608
Eigen values: [8.89

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 4.0531158447265625e-06
Solve time with direct kaug mode [s]: 0.8303799629211426
Total wall clock time [s]: 0.8310351371765137
Existed information has been added.
FIM: [[ 9.24366849e+13 -2.09987523e+13  1.91699340e+14 -1.78951508e+13
   7.89389757e+13 -1.38664256e+13]
 [-2.09987523e+13  6.11009774e+12 -4.25806959e+13  4.94718652e+12
  -1.72246220e+13  3.65611580e+12]
 [ 1.91699340e+14 -4.25806959e+13  4.03668984e+14 -3.70176131e+13
   1.68545308e+14 -2.92141232e+13]
 [-1.78951508e+13  4.94718652e+12 -3.70176131e+13  4.10193380e+12
  -1.52510820e+13  3.10380735e+12]
 [ 7.89389757e+13 -1.72246220e+13  1.68545308e+14 -1.52510820e+13
   7.12459702e+13 -1.22339553e+13]
 [-1.38664256e+13  3.65611580e+12 -2.92141232e+13  3.10380735e+12
  -1.22339553e+13  2.40179116e+12]]
Trace: 579965461859248.2
Determinant: 1.4567192492549018e+64
Condition number: 415269620.3891785
Minimal eigen value: 1385060.5910071135
Eigen values: [5.7517

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 4.76837158203125e-06
Solve time with direct kaug mode [s]: 0.5817508697509766
Total wall clock time [s]: 0.5823633670806885
Existed information has been added.
FIM: [[ 2.60128945e+16 -1.72020328e+16  6.00688473e+16 -1.66998207e+16
   2.71275925e+16 -1.44507801e+16]
 [-1.72020328e+16  1.13850643e+16 -3.97313609e+16  1.10533429e+16
  -1.79460695e+16  9.56518613e+15]
 [ 6.00688473e+16 -3.97313609e+16  1.38721938e+17 -3.85728400e+16
   6.26518641e+16 -3.33789817e+16]
 [-1.66998207e+16  1.10533429e+16 -3.85728400e+16  1.07315419e+16
  -1.74232635e+16  9.28687148e+15]
 [ 2.71275925e+16 -1.79460695e+16  6.26518641e+16 -1.74232635e+16
   2.82972002e+16 -1.50775195e+16]
 [-1.44507801e+16  9.56518613e+15 -3.33789817e+16  9.28687148e+15
  -1.50775195e+16  8.03678300e+15]]
Trace: 2.231854217084463e+17
Determinant: 1.15412707356528e+68
Condition number: 57096687331.47081
Minimal eigen value: 3908584.3898613085
Eigen values: [2.2316

In [8]:
fixed = {}

all_fim.extract_criteria()
print(all_fim.store_all_results_dataframe)

all_fim.figure_drawing(fixed, ['Temperature','x_R32'], 'PR quadratic','Temperature [K]', 'x (R32)' )


Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.1)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.2)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.3)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.4)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.5)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.6)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.7)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.8)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.9)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.1)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.2)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.3)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.4)
Design variable:  ['Temperature', 'x_R32']
Value          :  (28